# 1. Business Understanding

### 1. Project Overview

Tanzania is a country with a population of 57 million. It faces significant challenges in providing clean and reliable water to its population. 
The country has established numerous water points to meet this need but many of these water points are not fully functional, with some requiring repairs and others have failed entirely